## Build a database of stock data

This jupyter notebook will build a sqlite database from scratch using free data from Quandl. The db will contain tables for investor sentiment, company fundamentals (limited), various yield curves, new highs, new lows and adj ohlc and volume data for around 6000 US stocks.

Before you begin you will need to obtain a free api from Quandl: https://blog.quandl.com/getting-started-with-the-quandl-api

You will need the pandas and sqlalchemy libraries.

In [1]:
import pandas as pd
import quandl as q
from sqlalchemy import create_engine

# You must obtain your own free api key from Quandl
q.ApiConfig.api_key = '<your_api_key>'

# Create a sqlite db named 'jagged'
engine = create_engine('sqlite:///jagged.sqlite')

In [29]:
# Get the AAII sentiment index
sentiment = q.get("AAII/AAII_SENTIMENT")
sentiment.to_sql('sentiment',engine)

# get the free set of fundamental data from Zacks
fund = q.get_table("ZACKS/FE")
fund.to_sql('LimFundamentals',engine)

In [14]:
# Get Par Yield Curve, Instantaneous rate curve, One year forward rate curve and TIPS rate curve
par_yield_curve = q.get("FED/SVENPY")
inst_rt_curve = q.get("FED/SVENF")
one_yr_rt_curve = q.get("FED/SVEN1F")
tips_rt_curve = q.get("FED/TIPSY")

In [17]:
# Write to db
par_yield_curve.to_sql('YieldCurve',engine)
inst_rt_curve.to_sql('InstCurve',engine)
one_yr_rt_curve.to_sql('OneYearCurve',engine)
tips_rt_curve.to_sql('TIPSCurve',engine)

In [19]:
# Get New Highs, New Lows, Advancing volume, Declining volume from AMEX, NYSE, NASDAQ

amex_nh = q.get("URC/AMEX_52W_HI")
amex_nl = q.get("URC/AMEX_52W_LO")
ny_adv = q.get("URC/NYSE_ADV_VOL")
ny_dec = q.get("URC/NYSE_DEC_VOL")
nas_adv = q.get("URC/NASDAQ_ADV_VOL")
nas_dec = q.get("URC/NASDAQ_DEC_VOL")
am_adv = q.get("URC/AMEX_ADV_VOL")
am_dec = q.get("URC/AMEX_DEC_VOL")
ny_nh = q.get("URC/NYSE_52W_HI")
ny_nl = q.get("URC/NYSE_52W_LO")
nas_nh = q.get("URC/NASDAQ_52W_HI")
nas_nl = q.get("URC/NASDAQ_52W_LO")

In [23]:
# Write to db
amex_nh.to_sql('amex_nh',engine)
amex_nl.to_sql('amex_nl',engine)
ny_adv.to_sql('ny_adv',engine)
ny_dec.to_sql('ny_dec',engine)
nas_adv.to_sql('nas_adv',engine)
nas_dec.to_sql('nas_dec',engine)
am_adv.to_sql('am_adv',engine)
am_dec.to_sql('am_dec',engine)
ny_nh.to_sql('ny_nh',engine)
ny_nl.to_sql('nh_nl',engine)
nas_nh.to_sql('nas_nh',engine)
nas_nl.to_sql('nas_nl',engine)

In [2]:
# Obtained end of day stock prices as a csv from:
# "https://www.quandl.com/api/v3/datatables/WIKI/PRICES?qopts.export=true&api_key=<your_api_key>"
# Save the file to your local machine and enter the raw file path below

# Enter your file path here 
file = r'C:\Users\JoGa\Desktop\quandl\WIKI_PRICES.csv'
df = pd.read_csv(file)
print(len(df))
df.head()

14458978


,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
0,A,1999-11-18,45.50,50.00,40.00,44.00,44739900.0,0.0,1.0,43.365904,47.654840,38.123872,41.936259,44739900.0
1,A,1999-11-19,42.94,43.00,39.81,40.38,10897100.0,0.0,1.0,40.925976,40.983162,37.942784,38.486049,10897100.0
2,A,1999-11-22,41.31,44.00,40.06,44.00,4705200.0,0.0,1.0,39.372429,41.936259,38.181058,41.936259,4705200.0
3,A,1999-11-23,42.50,43.63,40.25,40.25,4274400.0,0.0,1.0,40.506614,41.583613,38.362146,38.362146,4274400.0
4,A,1999-11-24,40.13,41.94,40.00,41.06,3464400.0,0.0,1.0,38.247774,39.972880,38.123872,39.134155,3464400.0


In [3]:
# The dataframe is 14 million rows. You may want to increase or lower the chunksize 
# depending on the capabilities of your machine. Chunksize of 100,000 required about an
# additional 6GB of RAM. Will prob take a few min to complete.

df.to_sql('ohlc',engine,chunksize=100000)